<a href="https://colab.research.google.com/github/EduBrazGomes/ProjetoIntegrador/blob/main/Questao6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

In [10]:
df = pd.read_csv("precos_imoveis_com_m2.csv")
print(df.head(10))

  Localizacao  Quartos  Banheiros  Garagem  AnoConstrucao  Tamanho_m2  \
0  Zona Leste        4          4        1           2018         306   
1  Zona Oeste        1          4        1           1985         217   
2  Zona Norte        5          4        2           1996         292   
3  Zona Oeste        3          4        2           2004          95   
4  Zona Oeste        3          4        1           2007         122   
5    Zona Sul        1          1        1           1981         138   
6  Zona Norte        4          4        1           2014         188   
7  Zona Norte        4          1        2           2016         368   
8  Zona Norte        5          4        2           1984         195   
9  Zona Oeste        1          3        0           1980         229   

           Preco  
0  601125.067080  
1  352574.824604  
2  666303.643250  
3  396939.153972  
4  410361.614971  
5  297402.207803  
6  463612.099739  
7  603743.452177  
8  500853.033535  
9  322

In [11]:

df['idade'] = 2025 - df['AnoConstrucao']
df = df.drop('AnoConstrucao', axis=1)


In [12]:
x = df.drop('Preco', axis=1)

y = df['Preco']

categorical_features = ["Localizacao"]
numerical_features = ["Quartos","Banheiros","Garagem","idade","Tamanho_m2"]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [13]:
pipe_lr = Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())])
pipe_lr.fit(x_train, y_train)
y_pred_lr = pipe_lr.predict(x_test)

In [14]:
pipe_xgb = Pipeline(steps=[("preprocessor",preprocessor), ("regressor", xgb.XGBRegressor(objective="reg:squarederror",random_state=42))])
pipe_xgb.fit(x_train, y_train)
y_pred_xgb = pipe_xgb.predict(x_test)

In [15]:
pipe_mlp = Pipeline(steps=[("preprocessor",preprocessor), ("regressor", MLPRegressor(random_state=42, max_iter=1000))])
pipe_mlp.fit(x_train, y_train)
y_pred_mlp = pipe_mlp.predict(x_test)

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
def avaliar_modelo(y_true,y_pred, nome_modelo):
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_true, y_pred)
  print(f"Avaliação do modelo {nome_modelo}:")
  print(f"{nome_modelo} - RMSE: {rmse:.2}, R2: {r2:.3f}")

avaliar_modelo(y_test, y_pred_lr, "Regressão Linear")
avaliar_modelo(y_test, y_pred_xgb, "XGBoost")
avaliar_modelo(y_test, y_pred_mlp, "Rede neural (MLP)")


Avaliação do modelo Regressão Linear:
Regressão Linear - RMSE: 4.6e+04, R2: 0.878
Avaliação do modelo XGBoost:
XGBoost - RMSE: 5.6e+04, R2: 0.825
Avaliação do modelo Rede neural (MLP):
Rede neural (MLP) - RMSE: 4.6e+05, R2: -10.677
